In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../src')
from functions import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, chi2_contingency
from scipy.stats import ttest_ind

# Loading the Data:

In [2]:
df_clients_profile = pd.read_csv('../Data/Raw/df_final_demo.txt')
df_web_data_1 = pd.read_csv('../Data/Raw/df_final_web_data_pt_1.txt')
df_web_data_2 = pd.read_csv('../Data/Raw/df_final_web_data_pt_2.txt')
df_experiment_clients = pd.read_csv('../Data/Raw/df_final_experiment_clients.txt')
pd.set_option('display.max_columns', None) # To display all columns
df_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index= True)

# Exploratory Data Analysis (EDA):

## Initial Exploration

In [3]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,U,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,U,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [4]:
df_clients_profile.shape

(70609, 9)

In [5]:
df_experiment_clients.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
5,9466803,Test
6,3170143,Test
7,6651403,Control
8,5887227,Test
9,2105948,Control


In [6]:
df_experiment_clients.shape

(70609, 2)

In [7]:
df_web_data.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [8]:
df_web_data.shape

(755405, 5)

### Exploring numerical and categorical variables

In [9]:
# Retrieving the unique data types present in the dataframe columns
df_clients_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [10]:
df_experiment_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [11]:
df_web_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755405 entries, 0 to 755404
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     755405 non-null  int64 
 1   visitor_id    755405 non-null  object
 2   visit_id      755405 non-null  object
 3   process_step  755405 non-null  object
 4   date_time     755405 non-null  object
dtypes: int64(1), object(4)
memory usage: 28.8+ MB


In [12]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_profile = df_clients_profile.select_dtypes(include=[np.number]).columns
print(numerical_columns_profile)

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')


In [13]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_web = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_web)

Index(['client_id'], dtype='object')


In [14]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_exp_cli = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_exp_cli)

Index(['client_id'], dtype='object')


In [15]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_profile = df_clients_profile.select_dtypes("number").loc[:, df_clients_profile.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_profile

""
0
1
2
3
4
...
70604
70605
70606
70607


In [16]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_web= df_web_data.select_dtypes("number").loc[:, df_web_data.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_web

""
0
1
2
3
4
...
755400
755401
755402
755403


In [17]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_exp_cli= df_experiment_clients.select_dtypes("number").loc[:, df_experiment_clients.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_exp_cli

""
0
1
2
3
4
...
70604
70605
70606
70607


In [18]:
# Retrieving column names with object (typically string) data types from the dataframe
df_clients_profile.select_dtypes("object").columns

Index(['gendr'], dtype='object')

In [19]:
# Retrieving column names with object (typically string) data types from the dataframe
df_web_data.select_dtypes("object").columns

Index(['visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [20]:
# Retrieving column names with object (typically string) data types from the dataframe
df_experiment_clients.select_dtypes("object").columns

Index(['Variation'], dtype='object')

In [21]:
df_clients_profile.columns

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [22]:
df_clients_profile['gendr'] = df_clients_profile['gendr'].replace(['U', 'X'], 'Other')

## Drop duplicates values

In [23]:
#clean df_clients_profile
df_clients_profile = print_clean_data(df_clients_profile)

Before
client_id            0
clnt_tenure_yr      14
clnt_tenure_mnth    14
clnt_age            15
gendr               14
num_accts           14
bal                 14
calls_6_mnth        14
logons_6_mnth       14
dtype: int64
------------------------------------------
------------------------------------------
After
client_id           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            0
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
dtype: int64


In [24]:
#clean df_experiment_clients
df_experiment_clients = print_clean_data(df_experiment_clients)

Before
client_id        0
Variation    20109
dtype: int64
------------------------------------------
To many NaN values in column: Variation
------------------------------------------
After
client_id        0
Variation    20109
dtype: int64


In [25]:
#clean df_web_data
df_web_data = print_clean_data(df_web_data)

Before
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64
------------------------------------------
------------------------------------------
After
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [26]:
df_web_data.head(20)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


# Data Typing/Formatting

In [27]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,Other,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,Other,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,Other,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,Other,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,Other,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [28]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr      float64
clnt_tenure_mnth    float64
clnt_age            float64
gendr                object
num_accts           float64
bal                 float64
calls_6_mnth        float64
logons_6_mnth       float64
dtype: object

In [29]:
df_experiment_clients.dtypes

client_id     int64
Variation    object
dtype: object

In [30]:
df_web_data.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [31]:
df_web_data['date_time'] = pd.to_datetime(df_web_data['date_time'], format='%Y-%m-%d %H:%M:%S')

In [32]:
df_web_data.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [33]:
# Convert specified columns to integer data type
df_clients_profile['clnt_tenure_yr'] = df_clients_profile['clnt_tenure_yr'].astype(int)
df_clients_profile['clnt_tenure_mnth'] = df_clients_profile['clnt_tenure_mnth'].astype(int)
df_clients_profile['clnt_age'] = df_clients_profile['clnt_age'].astype(int)
df_clients_profile['num_accts'] = df_clients_profile['num_accts'].astype(int)
df_clients_profile['calls_6_mnth'] = df_clients_profile['calls_6_mnth'].astype(int)
df_clients_profile['logons_6_mnth'] = df_clients_profile['logons_6_mnth'].astype(int)

In [34]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age              int64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
dtype: object

## Create a data frame with the clients that won't participe in the experiment.

In [35]:
#setting aside the clients that wont be part if the experiment
null_df_experiment_clients = df_experiment_clients[df_experiment_clients['Variation'].isnull()]
null_df_experiment_clients

,client_id,Variation
50500,5459747,NaN
50501,8031000,NaN
50502,1847030,NaN
50503,9713157,NaN
50504,7775828,NaN
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


## Merge the clients in the web data

# df_test
### Identify the clients part of the test group

In [36]:
#id of clients that are part of the test
df_experiment_clients_Test = df_experiment_clients[df_experiment_clients['Variation'] == 'Test']
df_experiment_clients_Test.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
3,1982004,Test
5,9466803,Test
6,3170143,Test
8,5887227,Test
10,438567,Test
11,2519971,Test
13,5407849,Test
16,5308474,Test


### df_test: combine visit_id with visitor_id and reoganize the columns.

In [37]:
final_df_test = pd.merge(df_experiment_clients_Test, df_clients_profile, on='client_id', how='inner')
final_df_test = pd.merge(df_web_data, final_df_test, on='client_id', how='inner')
df_test = final_df_test
df_test.head(15)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5,64,79,Other,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5,64,79,Other,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5,64,79,Other,2,189023.86,1,4
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test,5,64,79,Other,2,189023.86,1,4
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test,5,64,79,Other,2,189023.86,1,4
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test,5,64,79,Other,2,189023.86,1,4
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,22,274,34,M,2,36001.90,5,8
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test,22,274,34,M,2,36001.90,5,8


### Create the csv file to df_test

In [38]:
df_test.to_csv('../Data/Cleaned_Data/df_test.csv', index=False)

# df_control
### Identify the clients part of the control group

In [39]:
#id of clients that are part of the test
df_experiment_clients_control = df_experiment_clients[df_experiment_clients['Variation'] == 'Control']
df_experiment_clients_control.head(10)

,client_id,Variation
2,4033851,Control
4,9294070,Control
7,6651403,Control
9,2105948,Control
12,9814849,Control
14,7286739,Control
15,4446011,Control
20,4229407,Control
24,2842481,Control
27,1218661,Control


### df_control: combine visit_id with visitor_id and reoganize the columns.

In [40]:
final_df_control = pd.merge(df_experiment_clients_control, df_clients_profile, on='client_id', how='inner')
final_df_control = pd.merge(df_web_data, final_df_control, on='client_id', how='inner')
df_control = final_df_control
df_control.head(15)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,12,149,63,M,2,142642.26,5,8
1,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:01,Control,12,149,63,M,2,142642.26,5,8
2,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:28:52,Control,12,149,63,M,2,142642.26,5,8
3,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:26:08,Control,12,149,63,M,2,142642.26,5,8
4,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:24:43,Control,12,149,63,M,2,142642.26,5,8
5,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:24:12,Control,12,149,63,M,2,142642.26,5,8
6,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:24:04,Control,12,149,63,M,2,142642.26,5,8
7,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
8,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
9,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:25,Control,12,149,63,M,2,142642.26,5,8


### Create the csv file to df_control

In [41]:
df_control.to_csv('../Data/Cleaned_Data/df_control.csv', index=False)

# Combine df_test and df_control

In [42]:
df_combined = pd.concat([df_test, df_control]).reset_index(drop=True)
df_combined.to_csv('../Data/Cleaned_Data/df_combined.csv', index=False)

# Pivot and create new columns

# df_test to df_test_final

In [43]:
df_test.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4


In [44]:
# Assuming df_control is your DataFrame
# 1. Data Preparation:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
df_test['date'] = df_test['date_time'].dt.date
df_test['visit_visitor_id'] = df_test['visit_id'].astype(str) + "_" + df_test['visitor_id'].astype(str)

In [45]:
# 2. Calculate time differences per step and make them positive:
df_test = df_test.sort_values(by=['visit_id', 'client_id', 'date_time'])
df_test['next_date_time'] = df_test.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)
df_test['time_diff_seconds'] = (df_test['next_date_time'] - df_test['date_time']).dt.total_seconds()
df_test['time_diff_minutes'] = df_test['time_diff_seconds'] / 60

# Get Last Step:
last_step_df = df_test.groupby('visit_visitor_id')['process_step'].last().reset_index()
last_step_df = last_step_df.rename(columns={'process_step': 'last_step'})

In [46]:
# 3. Calculate the number of steps per visit:
df_test = pd.concat([df_test, pd.get_dummies(df_test['process_step'], prefix='count')], axis=1)
df_steps_sum = df_test.groupby(by=['client_id', 'visit_id', 'visitor_id'])[['count_confirm', 
'count_start', 'count_step_1', 'count_step_2', 'count_step_3']].agg('sum')

In [47]:
# 4. Pivot the table to have one row per visit:
df_pivot = df_test.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None
test_time_counts = pd.merge(df_pivot, df_steps_sum, on=['client_id', 'visitor_id', 'visit_id'])


In [48]:
# 5. Merge the data with the clients' profile and get initial and final dates:
final_df_Test = pd.merge(test_time_counts, df_experiment_clients_Test, on='client_id', how='inner')
final_df_Test = pd.merge(final_df_Test, df_clients_profile, on='client_id', how='inner')

df_test_date = df_web_data.groupby(by = ['client_id', 'visit_id', 'visitor_id']).agg({'date_time': ['min', 'max']})
df_test_date.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_test_date.columns]
final_df_Test = pd.merge(df_test_date, final_df_Test, on=['client_id', 'visit_id', 'visitor_id'], how='inner')

# Ensure visit_visitor_id exists in final_df_Test for the merge:
final_df_Test['visit_visitor_id'] = final_df_Test['visit_id'].astype(str) + "_" + final_df_Test['visitor_id'].astype(str)  

# Calculate total time spent in the website:
final_df_Test['total_time_visit'] = (final_df_Test['date_timemax'] - final_df_Test['date_timemin']).dt.total_seconds() / 60

# Merge with last_step_df:
final_df_Test = pd.merge(final_df_Test, last_step_df, on='visit_visitor_id', how='left')

# Extract the date from date_timemax into a new column called date
final_df_Test['date'] = final_df_Test['date_timemax'].dt.date

# Remove unnecessary columns 
final_df_Test = final_df_Test.drop(columns=['visit_id', 'visitor_id'])

# Rename columns:
final_df_Test = final_df_Test.rename(columns={
    'date_timemin': 'initial_date', 
    'date_timemax': 'final_date', 
    'start': 'start_time', 
    'confirm': 'time_completion', 
    'count_confirm': 'completion',
    'count_start': 'start_step', 
    'count_step_1': '1st_step', 
    'count_step_2': '2nd_step', 
    'count_step_3': '3rd_step',
    'Variation': 'variation'
})

# Total navigations between start and last step (including start and last):
final_df_Test['navigations_bt_start_last'] = final_df_Test.apply(
    lambda row: row['start_step'] + row['1st_step'] + row['2nd_step'] + row['3rd_step'] + 1, 
    axis=1
)


In [49]:
# 6. Create the final order of columns:
new_order = ['client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3', 'time_completion', 'navigations_bt_start_last', 
             'completion', 'start_step', '1st_step', '2nd_step', '3rd_step', 'last_step', 'variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 
             'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth', 'date', 'initial_date', 'total_time_visit', 'final_date']
df_test_final = final_df_Test[new_order]

# Save the final DataFrame to CSV
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

# Convert data types if necessary
df_test_final = convert_data_types_final(df_test_final)
df_test_final.dtypes

client_id                            object
visit_visitor_id                     object
start_time                          float64
step_1                              float64
step_2                              float64
step_3                              float64
time_completion                     float64
navigations_bt_start_last             int64
completion                             bool
start_step                            int64
1st_step                              int64
2nd_step                              int64
3rd_step                              int64
last_step                            object
variation                            object
clnt_tenure_yr                        int64
clnt_tenure_mnth                      int64
clnt_age                              int64
gendr                                object
num_accts                             int64
bal                                 float64
calls_6_mnth                          int64
logons_6_mnth                   

In [50]:
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

In [51]:
df_test_final = convert_data_types_final(df_test_final)

In [52]:
df_test_final.dtypes

client_id                            object
visit_visitor_id                     object
start_time                          float64
step_1                              float64
step_2                              float64
step_3                              float64
time_completion                     float64
navigations_bt_start_last             int64
completion                             bool
start_step                            int64
1st_step                              int64
2nd_step                              int64
3rd_step                              int64
last_step                            object
variation                            object
clnt_tenure_yr                        int64
clnt_tenure_mnth                      int64
clnt_age                              int64
gendr                                object
num_accts                             int64
bal                                 float64
calls_6_mnth                          int64
logons_6_mnth                   

### Create the csv file to df_test_final

In [53]:
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

# df_control to df_control_final

In [54]:
# 1. Data Preparation
df_control['date_time'] = pd.to_datetime(df_control['date_time'])
df_control['date'] = df_control['date_time'].dt.date
df_control['visit_visitor_id'] = df_control['visit_id'].astype(str) + "_" + df_control['visitor_id'].astype(str)

In [55]:
# 2. Calculate time differences per step and make them positive:
df_control = df_control.sort_values(by=['visit_id', 'client_id', 'date_time'])
df_control['next_date_time'] = df_control.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)
df_control['time_diff_seconds'] = (df_control['next_date_time'] - df_control['date_time']).dt.total_seconds()
df_control['time_diff_minutes'] = df_control['time_diff_seconds'] / 60

# Get Last Step:
last_step_df_control = df_control.groupby('visit_visitor_id')['process_step'].last().reset_index()
last_step_df_control = last_step_df_control.rename(columns={'process_step': 'last_step'})


In [56]:
# 3. Calculate the number of steps per visit:
df_control = pd.concat([df_control, pd.get_dummies(df_control['process_step'], prefix='count')], axis=1)
df_steps_sum_control = df_control.groupby(by=['client_id', 'visit_id', 'visitor_id'])[['count_confirm', 
'count_start', 'count_step_1', 'count_step_2', 'count_step_3']].agg('sum')

In [57]:
# 4. Pivot the table to have one row per visit:
df_pivot_control = df_control.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot_control = df_pivot_control.reset_index()
df_pivot_control.columns.name = None
control_time_counts = pd.merge(df_pivot_control, df_steps_sum_control, on=['client_id', 'visitor_id', 'visit_id'])

In [58]:
# 4. Pivot the table to have one row per visit:
df_pivot_control = df_control.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot_control = df_pivot_control.reset_index()
df_pivot_control.columns.name = None
control_time_counts = pd.merge(df_pivot_control, df_steps_sum_control, on=['client_id', 'visitor_id', 'visit_id'])

In [59]:
# 5. Merge the data with the clients' profile and get initial and final dates:
final_df_Control = pd.merge(control_time_counts, df_experiment_clients, on='client_id', how='inner')
final_df_Control = pd.merge(final_df_Control, df_clients_profile, on='client_id', how='inner')

df_control_date = df_web_data.groupby(by = ['client_id', 'visit_id', 'visitor_id']).agg({'date_time': ['min', 'max']})
df_control_date.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_control_date.columns]
final_df_Control = pd.merge(df_control_date, final_df_Control, on=['client_id', 'visit_id', 'visitor_id'], how='inner')

# Ensure visit_visitor_id exists in final_df_Control for the merge:
final_df_Control['visit_visitor_id'] = final_df_Control['visit_id'].astype(str) + "_" + final_df_Control['visitor_id'].astype(str)  

# Calculate total time spent in the website:
final_df_Control['total_time_visit'] = (final_df_Control['date_timemax'] - final_df_Control['date_timemin']).dt.total_seconds() / 60

# Merge with last_step_df_control:
final_df_Control = pd.merge(final_df_Control, last_step_df_control, on='visit_visitor_id', how='left')

# Extract the date from date_timemax into a new column called date
final_df_Control['date'] = final_df_Control['date_timemax'].dt.date

# Remove unnecessary columns 
final_df_Control = final_df_Control.drop(columns=['visit_id', 'visitor_id'])

# Rename columns:
final_df_Control = final_df_Control.rename(columns={
    'date_timemin': 'initial_date', 
    'date_timemax': 'final_date', 
    'start': 'start_time', 
    'confirm': 'time_completion', 
    'count_confirm': 'completion',
    'count_start': 'start_step', 
    'count_step_1': '1st_step', 
    'count_step_2': '2nd_step', 
    'count_step_3': '3rd_step',
    'Variation': 'variation'
})

# Total navigations between start and last step (including start and last):
final_df_Control['navigations_bt_start_last'] = final_df_Control['start_step'] + final_df_Control['1st_step'] + final_df_Control['2nd_step'] + final_df_Control['3rd_step'] + 1

In [60]:
# 6. Create the final order of columns:
new_order_control = ['client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3', 'time_completion', 'navigations_bt_start_last', 
                    'completion', 'start_step', '1st_step', '2nd_step', '3rd_step', 'last_step', 'variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 
                    'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth', 'date', 'initial_date', 'total_time_visit', 'final_date']
df_control_final = final_df_Control[new_order_control]

# Save the final DataFrame to CSV
df_control_final.to_csv('../Data/Cleaned_Data/df_control_final.csv', index=False)

# Convert data types if necessary
df_control_final = convert_data_types_final(df_control_final)

# New Data Frames

## df_control_final

In [61]:
df_control_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,time_completion,navigations_bt_start_last,completion,start_step,1st_step,2nd_step,3rd_step,last_step,variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,date,initial_date,total_time_visit,final_date
0,1028,557292053_87239438319_391157_42237450_62128060588,0.82,1.57,4.73,1.85,0.0,10,False,1,5,2,1,step_1,Control,12,145,36,M,3,103520.22,1,4,2017-04-08,2017-04-08 18:51:28,8.97,2017-04-08 19:00:26
1,1104,543158812_46395476577_767725_194240915_1815800...,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,5,66,48,Other,3,154643.94,6,9,2017-06-12,2017-06-12 07:49:18,0.00,2017-06-12 07:49:18
2,1104,643221571_99977972121_69283_194240915_18158000533,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,5,66,48,Other,3,154643.94,6,9,2017-06-20,2017-06-20 22:31:33,0.00,2017-06-20 22:31:33
3,1186,507052512_11309370126_442139_446844663_3161510...,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,8,99,22,Other,2,31662.52,0,3,2017-04-08,2017-04-08 15:59:16,0.00,2017-04-08 15:59:16
4,1186,795373564_99931517312_810896_446844663_3161510...,0.18,0.18,0.00,0.00,0.0,4,False,1,1,1,0,step_2,Control,8,99,22,Other,2,31662.52,0,3,2017-04-08,2017-04-08 18:05:02,0.37,2017-04-08 18:05:24
5,1195,393817425_39015278493_996341_766842522_6999255...,0.55,1.63,0.52,1.38,0.0,5,True,1,1,1,1,confirm,Control,21,262,54,M,2,28457.96,2,5,2017-04-05,2017-04-05 20:15:26,4.08,2017-04-05 20:19:31
6,1197,71862471_21202285428_848395_753759429_54481946928,0.07,0.10,0.95,0.47,0.0,7,True,1,1,2,2,confirm,Control,7,85,28,Other,5,177062.14,5,8,2017-04-01,2017-04-01 17:16:24,1.58,2017-04-01 17:17:59
7,1368,784065271_45379483290_309335_366307863_1901466...,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,5,60,45,F,3,61956.25,6,9,2017-04-09,2017-04-09 21:58:28,0.00,2017-04-09 21:58:28
8,2439,848231744_22569944243_37711_607208067_70160939111,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,11,142,56,M,2,504142.55,0,3,2017-03-23,2017-03-23 14:59:23,0.00,2017-03-23 14:59:23
9,2581,182925466_27021409208_83502_770616558_80928163524,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,4,58,22,M,2,24976.79,1,4,2017-04-09,2017-04-09 06:11:41,0.00,2017-04-09 06:11:41


## df_test_final

In [62]:
df_test_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,time_completion,navigations_bt_start_last,completion,start_step,1st_step,2nd_step,3rd_step,last_step,variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,date,initial_date,total_time_visit,final_date
0,555,637149525_38041617439_716659_402506806_5608737...,0.12,0.53,1.65,0.33,0.00,5,True,1,1,1,1,confirm,Test,3,46,29,Other,2,25454.66,2,6,2017-04-15,2017-04-15 12:57:56,2.63,2017-04-15 13:00:34
1,647,40369564_40101682850_311847_66758770_53988066587,0.12,0.30,3.15,2.72,0.00,5,True,1,1,1,1,confirm,Test,12,151,57,M,2,30525.80,0,4,2017-04-12,2017-04-12 15:41:28,6.28,2017-04-12 15:47:45
2,934,7076463_57954418406_971348_810392784_45004760546,2.37,0.00,0.00,0.00,0.00,5,False,4,0,0,0,start,Test,9,109,51,F,2,32522.88,0,3,2017-04-18,2017-04-18 02:36:30,2.37,2017-04-18 02:38:52
3,1336,583743392_96265099036_939815_920624746_3260333...,0.42,0.43,0.85,1.82,0.00,5,True,1,1,1,1,confirm,Test,48,576,42,M,4,130537.18,6,9,2017-05-08,2017-05-08 06:05:12,3.52,2017-05-08 06:08:43
4,1336,614001770_19101025926_112779_920624746_3260333...,0.00,0.00,0.00,0.00,1.37,1,True,0,0,0,0,confirm,Test,48,576,42,M,4,130537.18,6,9,2017-05-08,2017-05-08 08:21:38,1.37,2017-05-08 08:23:00
5,1346,27144337_83739845380_214282_123474046_4204671056,2.65,3.37,0.82,1.42,0.00,11,False,5,2,1,2,step_3,Test,14,177,46,F,2,822512.91,3,6,2017-06-06,2017-06-06 18:20:28,8.25,2017-06-06 18:28:43
6,1516,255400977_38039535960_779641_182314299_6316858...,0.93,0.58,10.12,4.27,0.00,5,True,1,1,1,1,confirm,Test,12,150,58,F,2,25408.39,5,8,2017-04-06,2017-04-06 22:14:24,15.90,2017-04-06 22:30:18
7,1516,481123290_95510977345_707323_182314299_6316858...,1.33,1.67,1.08,0.00,0.00,12,False,5,3,3,0,step_1,Test,12,150,58,F,2,25408.39,5,8,2017-04-05,2017-04-05 15:53:31,4.08,2017-04-05 15:57:36
8,1643,379946188_1773022140_107963_910842861_51062379179,0.00,0.00,0.00,0.00,0.00,2,False,1,0,0,0,start,Test,19,229,55,M,2,81731.39,0,3,2017-04-15,2017-04-15 15:06:00,0.00,2017-04-15 15:06:00
9,1643,633860590_96880450633_976109_910842861_5106237...,1.47,0.75,0.00,0.00,0.00,5,False,3,1,0,0,start,Test,19,229,55,M,2,81731.39,0,3,2017-04-24,2017-04-24 06:36:07,2.22,2017-04-24 06:38:20


### Create the csv file to combine_df

In [63]:
df_final = pd.concat([df_test_final, df_control_final], ignore_index=True)

In [64]:
df_final = convert_data_types(df_final)

In [65]:
df_final.to_csv('../Data/Cleaned_Data/df_final.csv', index=False)

In [66]:
df_final.sample(15)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,time_completion,navigations_bt_start_last,completion,start_step,1st_step,2nd_step,3rd_step,last_step,variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,date,initial_date,total_time_visit,final_date
38263,334941,203150362_77507753878_503268_772898027_1918013...,0.37,0.10,1.00,0.97,0.0,5,True,1,1,1,1,confirm,Control,6,78,46,Other,2,70433.97,1,4,2017-04-05,2017-04-05 09:57:32,2.43,2017-04-05 09:59:58
29349,7879083,324373623_6616351181_132698_707993386_30845727112,0.78,0.00,0.00,0.00,0.0,3,False,2,0,0,0,start,Test,4,57,65,F,2,122301.34,6,9,2017-03-30,2017-03-30 09:57:18,0.78,2017-03-30 09:58:05
13164,3571114,801005284_535612566_688713_997735941_39507794398,0.05,0.25,0.75,0.00,0.0,5,False,1,1,1,1,step_3,Test,5,65,27,Other,2,36888.60,6,9,2017-03-29,2017-03-29 11:07:04,1.05,2017-03-29 11:08:07
48619,3598499,859113892_1428236827_378138_648471056_22894118209,0.28,0.08,0.75,0.00,0.0,5,False,1,1,1,1,step_3,Control,6,74,27,Other,3,91577.67,3,6,2017-04-05,2017-04-05 11:07:13,1.12,2017-04-05 11:08:20
52305,4737509,119193083_64521116486_355765_935570566_6565269...,4.18,1.35,0.83,0.00,0.0,5,False,1,1,1,1,step_3,Control,21,254,57,F,2,205440.69,5,8,2017-03-29,2017-03-29 23:23:41,6.37,2017-03-29 23:30:03
42137,1573977,188230776_8948929833_80629_2613106_66087979197,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Control,5,63,58,F,2,333223.65,6,9,2017-04-01,2017-04-01 17:53:03,0.00,2017-04-01 17:53:03
23588,6347993,206312236_79706108718_940453_134042773_7948663...,0.30,0.10,0.83,0.30,0.0,5,True,1,1,1,1,confirm,Test,9,110,36,M,4,162509.13,6,9,2017-05-03,2017-05-03 08:08:43,1.53,2017-05-03 08:10:15
66390,9046844,403132737_98705539503_761452_278293848_5519057...,0.00,0.00,0.00,0.00,0.0,3,False,2,0,0,0,start,Control,6,72,36,Other,2,37389.75,2,5,2017-06-02,2017-06-02 13:11:31,0.00,2017-06-02 13:11:31
59868,7038864,470059346_99240485427_721576_656713130_2511496...,0.13,0.23,2.38,0.85,0.0,6,True,2,1,1,1,confirm,Control,21,261,26,F,3,111816.42,6,9,2017-04-27,2017-04-27 12:20:23,3.60,2017-04-27 12:23:59
13806,3732108,617421228_85564811434_723461_355002843_5870219...,0.00,0.00,0.00,0.00,0.0,2,False,1,0,0,0,start,Test,6,77,45,Other,2,31407.73,1,4,2017-06-08,2017-06-08 16:32:28,0.00,2017-06-08 16:32:28


In [67]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69425 entries, 0 to 69424
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   client_id                  69425 non-null  object        
 1   visit_visitor_id           69425 non-null  object        
 2   start_time                 69425 non-null  float64       
 3   step_1                     69425 non-null  float64       
 4   step_2                     69425 non-null  float64       
 5   step_3                     69425 non-null  float64       
 6   time_completion            69425 non-null  float64       
 7   navigations_bt_start_last  69425 non-null  int64         
 8   completion                 69425 non-null  bool          
 9   start_step                 69425 non-null  int64         
 10  1st_step                   69425 non-null  int64         
 11  2nd_step                   69425 non-null  int64         
 12  3rd_

In [68]:
df_control_final['last_step'].value_counts()

last_step
confirm    15383
start       9640
step_1      3502
step_3      2250
step_2      1460
Name: count, dtype: int64

In [69]:
df_final[['variation','last_step']].value_counts()

variation  last_step
Test       confirm      21496
Control    confirm      15383
           start         9640
Test       start         9452
Control    step_1        3502
Test       step_1        3139
Control    step_3        2250
Test       step_3        1782
Control    step_2        1460
Test       step_2        1321
Name: count, dtype: int64